<a href="https://colab.research.google.com/github/yuki-tamaribuchi/kaggle_codes/blob/master/kaggle_titanic_rbfkernel_and_sgd_clf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir /root/.kaggle
!cp drive/'My Drive'/kaggle.json /root/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [2]:
!kaggle competitions download -c titanic

train.csv: Skipping, found more recently modified local copy (use --force to force download)
gender_submission.csv: Skipping, found more recently modified local copy (use --force to force download)
test.csv: Skipping, found more recently modified local copy (use --force to force download)


In [3]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split

In [4]:
train_df=pd.read_csv('train.csv')
test_df=pd.read_csv('test.csv')

In [5]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
nulls=train_df.isnull().sum()

In [7]:
nulls

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [8]:
sex_dummy=pd.get_dummies(train_df['Sex'])
pclass_dummy=pd.get_dummies(train_df['Pclass'],prefix='Pclass')
embarked_dummy=pd.get_dummies(train_df['Embarked'],prefix='Emb')

train_df=train_df.drop(['Ticket','Cabin','Name','Sex','Pclass','Embarked','Cabin'],axis=1)
train_df=pd.concat([train_df,sex_dummy,pclass_dummy,embarked_dummy],axis=1)

train_df['Age']=train_df['Age'].fillna(train_df['Age'].mean())

train_df.head()

y_train=train_df['Survived']
X_train=train_df.drop(['Survived'],axis=1)

In [9]:
nulls=train_df.isnull().sum()

In [10]:
nulls

PassengerId    0
Survived       0
Age            0
SibSp          0
Parch          0
Fare           0
female         0
male           0
Pclass_1       0
Pclass_2       0
Pclass_3       0
Emb_C          0
Emb_Q          0
Emb_S          0
dtype: int64

In [11]:
X_train.head()

,PassengerId,Age,SibSp,Parch,Fare,female,male,Pclass_1,Pclass_2,Pclass_3,Emb_C,Emb_Q,Emb_S
0,1,22.0,1,0,7.2500,0,1,0,0,1,0,0,1
1,2,38.0,1,0,71.2833,1,0,1,0,0,1,0,0
2,3,26.0,0,0,7.9250,1,0,0,0,1,0,0,1
3,4,35.0,1,0,53.1000,1,0,1,0,0,0,0,1
4,5,35.0,0,0,8.0500,0,1,0,0,1,0,0,1


In [12]:
y_train.head()

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

In [13]:
nulls=test_df.isnull().sum()

In [14]:
nulls

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [15]:
sex_dummy=pd.get_dummies(test_df['Sex'])
pclass_dummy=pd.get_dummies(test_df['Pclass'],prefix='Pclass')
embarked_dummy=pd.get_dummies(test_df['Embarked'],prefix='Emb')

test_df=test_df.drop(['Ticket','Cabin','Name','Sex','Pclass','Embarked','Cabin'],axis=1)
test_df=pd.concat([test_df,sex_dummy,pclass_dummy,embarked_dummy],axis=1)

test_df['Age']=test_df['Age'].fillna(test_df['Age'].mean())
test_df['Fare']=test_df['Fare'].fillna(test_df['Fare'].mean())

test_df.head()

X_test_for_pred=test_df

In [16]:
nulls=X_test_for_pred.isnull().sum()

In [17]:
nulls

PassengerId    0
Age            0
SibSp          0
Parch          0
Fare           0
female         0
male           0
Pclass_1       0
Pclass_2       0
Pclass_3       0
Emb_C          0
Emb_Q          0
Emb_S          0
dtype: int64

In [18]:
X_train,X_test,y_train,y_test=train_test_split(X_train,y_train,test_size=0.2)

In [19]:
from sklearn.kernel_approximation import RBFSampler
from sklearn.linear_model import SGDClassifier

In [20]:
rbf_feature=RBFSampler()
X_features=rbf_feature.fit_transform(X_train)
clf=SGDClassifier()
clf.fit(X_features,y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [21]:
X_test_features=rbf_feature.transform(X_test)
clf.score(X_test_features,y_test)

0.5307262569832403

In [22]:
X_test_features=rbf_feature.transform(X_test_for_pred)
y_pred=clf.predict(X_test_features)

In [23]:
y_pred_df=pd.DataFrame(y_pred)
y_pred_df=y_pred_df.rename(columns={0:'Survived'})

id=pd.DataFrame(X_test['PassengerId'])

result=pd.concat([id,y_pred_df],axis=1)

In [24]:
result

,PassengerId,Survived
0,NaN,0.0
1,NaN,1.0
2,NaN,0.0
3,NaN,0.0
4,NaN,0.0
...,...,...
866,867.0,NaN
869,870.0,NaN
870,871.0,NaN
872,873.0,NaN


In [25]:
result.to_csv('submission.csv',index=False)

In [26]:
#!kaggle competitions submit -c titanic -f submission.csv -m "Titanic Predict with RBFKernel and SGDclf"